In [1]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
import numpy as np

In [2]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # ядро матрицы 3 на 3
pool_size = 2 # уменьшение масштаба на 2 то есть исходная матрица изображения уменьшается на 2 по вертикали и горизонтали (пример в начале методички)
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer... количество ядер на 1 сверточном
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer на 2 сверточном
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25 вероятность удаления нейронов дропаутом
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data

num_train, depth, height, width = X_train.shape # there are 50000 training examples in CIFAR-10 
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_train) # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

In [4]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes firstin Keras
# Conv [32] -> Conv [32] -> Pool (with dropout on the poolinglayer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size, pool_size, padding='same')(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the poolinglayer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size,padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size,padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size, pool_size, padding='same')(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply Dense -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)
model = Model(inp, out) # To define a model, justspecify its input and output layers
model.compile(loss='categorical_crossentropy', # using thecross-entropy loss function

optimizer='adam', # using the Adam optimiser
metrics=['accuracy']) # reporting the accuracy
model.fit(X_train, Y_train, batch_size, num_epochs, verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trainedmodel on the test set!

Epoch 1/200
1407/1407 [==============================] - 10s 6ms/step - loss: 1.9219 - accuracy: 0.2739 - val_loss: 1.6717 - val_accuracy: 0.3754
Epoch 2/200
1407/1407 [==============================] - 8s 6ms/step - loss: 1.7095 - accuracy: 0.3668 - val_loss: 1.6033 - val_accuracy: 0.4008
Epoch 3/200
1407/1407 [==============================] - 8s 6ms/step - loss: 1.6413 - accuracy: 0.3952 - val_loss: 1.5299 - val_accuracy: 0.4370
Epoch 4/200
1407/1407 [==============================] - 8s 6ms/step - loss: 1.6007 - accuracy: 0.4136 - val_loss: 1.5241 - val_accuracy: 0.4432
Epoch 5/200
1407/1407 [==============================] - 8s 6ms/step - loss: 1.5642 - accuracy: 0.4311 - val_loss: 1.4547 - val_accuracy: 0.4676
Epoch 6/200
1407/1407 [==============================] - 8s 6ms/step - loss: 1.5366 - accuracy: 0.4438 - val_loss: 1.4369 - val_accuracy: 0.4718
Epoch 7/200
1407/1407 [==============================] - 8s 6ms/step - loss: 1.5129 - accuracy: 0.4505 - val_loss: 1.4033 - val_a

[1841.1876220703125, 0.16449999809265137]

In [9]:
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from tensorflow.keras.optimizers import SGD


def create_model():
    model = Sequential()
    model.add(Conv2D(32, 3, 3, activation='relu', padding='same'))
    model.add(Conv2D(32, 3, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2, padding='same'))
#    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, 3, 3, activation='relu', padding='same'))
    model.add(Conv2D(64, 3, 3, activation='relu', padding='same'))
    model.add(MaxPooling2D(2, 2, padding='same'))
#    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
#    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    # compile model
    #opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model2 = create_model()
# fit model
model2.fit(X_train, Y_train, epochs=200, batch_size=32, validation_split=0.1, verbose=1)

model2.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

1407/1407 [==============================] - 8s 5ms/step - loss: 0.5541 - accuracy: 0.7935 - val_loss: 2.1059 - val_accuracy: 0.5430
Epoch 80/200
1407/1407 [==============================] - 7s 5ms/step - loss: 0.5552 - accuracy: 0.7935 - val_loss: 2.1051 - val_accuracy: 0.5380
Epoch 81/200
1407/1407 [==============================] - 7s 5ms/step - loss: 0.5486 - accuracy: 0.7942 - val_loss: 2.0740 - val_accuracy: 0.5380
Epoch 82/200
1407/1407 [==============================] - 7s 5ms/step - loss: 0.5381 - accuracy: 0.7978 - val_loss: 2.1927 - val_accuracy: 0.5396
Epoch 83/200
1407/1407 [==============================] - 7s 5ms/step - loss: 0.5484 - accuracy: 0.7938 - val_loss: 2.1305 - val_accuracy: 0.5384
Epoch 84/200
1407/1407 [==============================] - 7s 5ms/step - loss: 0.5424 - accuracy: 0.7986 - val_loss: 2.1835 - val_accuracy: 0.5396
Epoch 85/200
1407/1407 [==============================] - 7s 5ms/step - loss: 0.5318 - accuracy: 0.7999 - val_loss: 2.1155 - val_accuracy

[1794.995361328125, 0.2980000078678131]

In [10]:
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from tensorflow.keras.optimizers import SGD


model3 = Sequential()
model3.add(Conv2D(32, 5, 5, activation='relu', padding='same'))
model3.add(Conv2D(32, 5, 5, activation='relu', padding='same'))
model3.add(MaxPooling2D((2, 2), padding='same'))
model3.add(Dropout(0.25))
    
model3.add(Conv2D(64, 5, 5, activation='relu', padding='same'))
model3.add(Conv2D(64, 5, 5, activation='relu', padding='same'))
model3.add(MaxPooling2D((2, 2), padding='same'))
model3.add(Dropout(0.25))
    
model3.add(Flatten())
model3.add(Dense(512, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(10, activation='softmax'))

model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model3.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

model3.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Epoch 1/50
1407/1407 [==============================] - 9s 6ms/step - loss: 2.0039 - accuracy: 0.2220 - val_loss: 1.7694 - val_accuracy: 0.3266
Epoch 2/50
1407/1407 [==============================] - 8s 6ms/step - loss: 1.7667 - accuracy: 0.3445 - val_loss: 1.6264 - val_accuracy: 0.4048
Epoch 3/50
1407/1407 [==============================] - 8s 6ms/step - loss: 1.6672 - accuracy: 0.3894 - val_loss: 1.5241 - val_accuracy: 0.4420
Epoch 4/50
1407/1407 [==============================] - 8s 6ms/step - loss: 1.6108 - accuracy: 0.4110 - val_loss: 1.4775 - val_accuracy: 0.4560
Epoch 5/50
1407/1407 [==============================] - 8s 6ms/step - loss: 1.5714 - accuracy: 0.4249 - val_loss: 1.4368 - val_accuracy: 0.4708
Epoch 6/50
1407/1407 [==============================] - 8s 6ms/step - loss: 1.5313 - accuracy: 0.4414 - val_loss: 1.4074 - val_accuracy: 0.4826
Epoch 7/50
1407/1407 [==============================] - 8s 6ms/step - loss: 1.5077 - accuracy: 0.4519 - val_loss: 1.4047 - val_accuracy:

[151.5417938232422, 0.31520000100135803]

In [13]:
model4 = Sequential()
model4.add(Conv2D(32, (4,4), activation='relu', padding='same'))
model4.add(Conv2D(32, (4,4), activation='relu', padding='same'))
model4.add(MaxPooling2D((2, 2), padding='same'))
model4.add(Dropout(0.25))
    
model4.add(Conv2D(64, (4,4), activation='relu', padding='same'))
model4.add(Conv2D(64, (4,4), activation='relu', padding='same'))
model4.add(MaxPooling2D((2, 2), padding='same'))
model4.add(Dropout(0.25))
    
model4.add(Flatten())
model4.add(Dense(512, activation='relu'))
model4.add(Dropout(0.5))
model4.add(Dense(10, activation='softmax'))

model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model4.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

model4.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Epoch 1/50
1407/1407 [==============================] - 260s 184ms/step - loss: 1.6167 - accuracy: 0.4060 - val_loss: 1.2315 - val_accuracy: 0.5500
Epoch 2/50
1407/1407 [==============================] - 257s 182ms/step - loss: 1.2435 - accuracy: 0.5514 - val_loss: 1.1006 - val_accuracy: 0.6066
Epoch 3/50
1407/1407 [==============================] - 242s 172ms/step - loss: 1.0622 - accuracy: 0.6222 - val_loss: 0.9006 - val_accuracy: 0.6922
Epoch 4/50
1407/1407 [==============================] - 226s 161ms/step - loss: 0.9590 - accuracy: 0.6607 - val_loss: 0.8452 - val_accuracy: 0.7104
Epoch 5/50
1407/1407 [==============================] - 226s 161ms/step - loss: 0.8867 - accuracy: 0.6847 - val_loss: 0.7532 - val_accuracy: 0.7402
Epoch 6/50
1407/1407 [==============================] - 243s 173ms/step - loss: 0.8367 - accuracy: 0.7070 - val_loss: 0.7435 - val_accuracy: 0.7442
Epoch 7/50
1407/1407 [==============================] - 251s 178ms/step - loss: 0.8000 - accuracy: 0.7175 - val_

[386.2386779785156, 0.5551000237464905]

In [14]:
model4 = Sequential()
model4.add(Conv2D(32, (4,4), activation='relu', padding='same'))
model4.add(Conv2D(32, (4,4), activation='relu', padding='same'))
model4.add(MaxPooling2D((2, 2), padding='same'))
model4.add(Dropout(0.25))
    
model4.add(Conv2D(64, (4,4), activation='relu', padding='same'))
model4.add(Conv2D(64, (4,4), activation='relu', padding='same'))
model4.add(MaxPooling2D((2, 2), padding='same'))
model4.add(Dropout(0.25))
    
model4.add(Flatten())
model4.add(Dense(512, activation='relu'))
model4.add(Dropout(0.5))
model4.add(Dense(10, activation='softmax'))

model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model4.fit(X_train, Y_train, epochs=20, batch_size=32, validation_split=0.1, verbose=1)

model4.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Epoch 1/20
1407/1407 [==============================] - 256s 182ms/step - loss: 1.5959 - accuracy: 0.4155 - val_loss: 1.2317 - val_accuracy: 0.5544
Epoch 2/20
1407/1407 [==============================] - 253s 180ms/step - loss: 1.1875 - accuracy: 0.5764 - val_loss: 0.9581 - val_accuracy: 0.6648
Epoch 3/20
1407/1407 [==============================] - 257s 182ms/step - loss: 1.0104 - accuracy: 0.6417 - val_loss: 0.8839 - val_accuracy: 0.6936
Epoch 4/20
1407/1407 [==============================] - 253s 179ms/step - loss: 0.9179 - accuracy: 0.6750 - val_loss: 0.8188 - val_accuracy: 0.7130
Epoch 5/20
1407/1407 [==============================] - 236s 168ms/step - loss: 0.8454 - accuracy: 0.7021 - val_loss: 0.7716 - val_accuracy: 0.7382
Epoch 6/20
1407/1407 [==============================] - 247s 175ms/step - loss: 0.7886 - accuracy: 0.7223 - val_loss: 0.7285 - val_accuracy: 0.7488
Epoch 7/20
1407/1407 [==============================] - 257s 183ms/step - loss: 0.7376 - accuracy: 0.7401 - val_

[291.6810607910156, 0.5379999876022339]